## Setup

Please execute the cell(s) below to initialize the notebook environment.

In [1]:
# Imports
on_colab = 'google.colab' in str(get_ipython())

import warnings
warnings.filterwarnings('ignore')

if not on_colab:
  import napari
import smorph as sm
import smorph.util.autocrop as ac
import ipywidgets as widgets

from magicgui import magicgui
import winsound

In [2]:
# Helper function
def view_3D(*args):
    """Views a 3D image."""
    n_images = len(args)
    if not on_colab and n_images > 0:
        with napari.gui_qt():
            viewer = napari.view_image(**args[0], ndisplay=3, scale=pipe.SCALE)
            for itr in range(1, n_images):
                if args[itr]['data'].max() > 1 and args[itr]['data'].max() % 1 == 0:
                    if 'colormap' in args[itr].keys():
                        del args[itr]['colormap']
                    if 'gamma' in args[itr].keys():
                        del args[itr]['gamma']
                    viewer.add_labels(**args[itr], scale=pipe.SCALE)
                else:
                    viewer.add_image(**args[itr], scale=pipe.SCALE)

---

## Step 1: Import Microscopic Image of the Tissue

Set `TISSUE_IMAGE` to the path of the image file to be processed.

1. Deconvolution
2. Rolling ball background subtraction
3. CLAHE
4. ROI selection
5. Non-local means denoising using auto-calibrated parameters using J-Invariance

In [3]:
TISSUE_IMAGE = 'Datasets/Confocal/SAL,DMI, FLX ADN HALO_TREATMENT_28 DAYS/control_28 days/CONTROL_MSP2.1MB_4_LONG MARK_20X_SEC 1_LEFT HILUS_28 DAYczi.czi'  #@param
FILE_ROI = 'Datasets/Confocal/SAL,DMI, FLX ADN HALO_TREATMENT_28 DAYS/control_28 days/CONTROL_MSP2.1MB_4_LONG MARK_20X_SEC 1_LEFT HILUS_28 DAYczi_ML.roi'
ROI_NAME = 'ML'

DECONV_ITR = 30
CLIP_LIMIT = .02

REF_IMAGE = 'Datasets/Confocal/SAL,DMI, FLX ADN HALO_TREATMENT_28 DAYS/control_28 days/CONTROL_MSP3.1M_1_SINGLE MARK_20X_SEC 1_RIGHT HILUS_28 DAYczi.czi'
REF_FILE_ROI = 'Datasets/Confocal/SAL,DMI, FLX ADN HALO_TREATMENT_28 DAYS/control_28 days/CONTROL_MSP3.1M_1_SINGLE MARK_20X_SEC 1_RIGHT HILUS_28 DAYczi_ML.roi'

cache_dir = 'Cache/'

pipe = ac.TissueImage(TISSUE_IMAGE, FILE_ROI, ROI_NAME, DECONV_ITR, CLIP_LIMIT, REF_IMAGE, REF_FILE_ROI, cache_dir)
winsound.MessageBeep(0)

---

## Step 2: Select ROI using Polygonal Lasso Tool

Set two variables:
- `SELECT_ROI`: True, If you want to select ROI manually; else False
- `NAME_ROI`: Name of the manually selected ROI
- `FILE_ROI`: Path to the ROI file; else None

In [ ]:
view_3D({'data': pipe.imoriginal, 'colormap': 'magma', 'name': 'original'},
        {'data': pipe.imdeconvolved, 'colormap': 'magma', 'name': 'deconvolved'},
        {'data': pipe.impreprocessed, 'colormap': 'magma', 'name': 'preprocessed'},
        {'data': pipe.imdenoised, 'colormap': 'magma', 'name': 'denoised'})

---

## Step 3: Segmentation

### 3.1 Global threshold & color label cells

Set two parameters:
- `LOW_THRESH`: Pixel intensity value corresponding to faintest branch's edge
- `HIGH_THRESH`: Pixel intensity value corresponding to faintest soma

Understand their effect by configuring three parameters:
- `LOW_DELTA`: Pixel intensity value corresponding to change in `LOW_THRESH`
- `HIGH_DELTA`: Pixel intensity value corresponding to change in `HIGH_THRESH`
- `N_STEPS`: Number of steps of delta in threshold to take in both directions

In [ ]:
LOW_THRESH = .07
HIGH_THRESH = .12
LOW_AUTO_THRESH = 'triangle'
HIGH_AUTO_THRESH = None

LOW_DELTA = .01
HIGH_DELTA = .01
N_STEPS = 0
results = None

import skimage
%matplotlib inline
def test_thresholds(low_thresh_init, low_thresh, high_thresh_init,
                    high_thresh, low_delta, high_delta, n_steps):
  global results, LOW_THRESH, HIGH_THRESH, LOW_DELTA, HIGH_DELTA, N_STEPS, LOW_AUTO_THRESH, HIGH_AUTO_THRESH
  LOW_THRESH, HIGH_THRESH, N_STEPS = low_thresh, high_thresh, n_steps
  if low_thresh_init is not None:
    LOW_THRESH = eval(f'skimage.filters.threshold_{low_thresh_init}(pipe.imdenoised[:, lly:ury, llx:urx])')
  if high_thresh_init is not None:
    HIGH_THRESH = eval(f'skimage.filters.threshold_{high_thresh_init}(pipe.imdenoised[:, lly:ury, llx:urx])')
  LOW_DELTA, HIGH_DELTA = low_delta, high_delta
  results = ac.testThresholds(pipe.imdenoised, LOW_THRESH, HIGH_THRESH, LOW_DELTA,
                              HIGH_DELTA, N_STEPS, 'gist_earth')

if on_colab:
  _ = widgets.interact(test_thresholds,
                      low_thresh_init=[None, *sm.util.THRESHOLD_METHODS],
                      low_thresh=widgets.FloatSlider(LOW_THRESH, min=0, max=1, step=.01,
                                                      readout_format='.4f', layout=widgets.Layout(width='100%')),
                      high_thresh_init=[None, *sm.util.THRESHOLD_METHODS],
                      high_thresh=widgets.FloatSlider(HIGH_THRESH, min=0, max=1, step=.01,
                                                      readout_format='.4f', layout=widgets.Layout(width='100%')),
                      low_delta=widgets.FloatSlider(LOW_DELTA, min=0, max=1, step=.0005,
                                                    readout_format='.4f', layout=widgets.Layout(width='100%')),
                      high_delta=widgets.FloatSlider(HIGH_DELTA, min=0, max=1, step=.0005,
                                                      readout_format='.4f', layout=widgets.Layout(width='100%')),
                      n_steps=widgets.IntSlider(N_STEPS, min=0, max=10,
                                                layout=widgets.Layout(width='100%'))
  )
  # view_3D({'data': pipe.imdenoised, 'colormap': 'gray_r', 'name': 'denoised'}, *results)
else:
  viewer = napari.Viewer(ndisplay=3)
  viewer.add_image(pipe.impreprocessed, scale=pipe.SCALE)
  @magicgui(
    call_button="Test Thresholds",
    low_auto_thresh={'choices': [None, *sm.util.THRESHOLD_METHODS]},
    low_thresh={"widget_type": "FloatSlider", 'max': 1},
    high_auto_thresh={'choices': [None, *sm.util.THRESHOLD_METHODS]},
    high_thresh={"widget_type": "FloatSlider", 'max': 1},
    low_delta={"widget_type": "FloatSlider", 'max': 1},
    high_delta={"widget_type": "FloatSlider", 'max': 1}
  )
  def test_thresholds_gui(
      low_auto_thresh=LOW_AUTO_THRESH,
      low_thresh=LOW_THRESH,
      high_auto_thresh=HIGH_AUTO_THRESH,
      high_thresh=HIGH_THRESH,
      low_delta=LOW_DELTA,
      high_delta=HIGH_DELTA,
      n_steps=N_STEPS
  ):
    global results, LOW_THRESH, HIGH_THRESH, LOW_DELTA, HIGH_DELTA, N_STEPS, LOW_AUTO_THRESH, HIGH_AUTO_THRESH
    LOW_THRESH, HIGH_THRESH, N_STEPS = low_thresh, high_thresh, n_steps
    LOW_AUTO_THRESH, HIGH_AUTO_THRESH = low_auto_thresh, high_auto_thresh
    lly, llx, ury, urx = pipe.in_box
    if low_auto_thresh is not None:
      LOW_THRESH = eval(f'skimage.filters.threshold_{low_auto_thresh}(pipe.imdenoised[:, lly:ury, llx:urx])')
    if high_auto_thresh is not None:
      HIGH_THRESH = eval(f'skimage.filters.threshold_{high_auto_thresh}(pipe.imdenoised[:, lly:ury, llx:urx])')
    LOW_DELTA, HIGH_DELTA = low_delta, high_delta
    results = ac.core._testThresholds(pipe.imdenoised, LOW_THRESH, HIGH_THRESH, LOW_DELTA,
                                      HIGH_DELTA, N_STEPS)
    past_state = viewer.window.qt_viewer.view.camera.get_state()
    viewer.layers.clear()

    viewer.add_image(pipe.impreprocessed, scale=pipe.SCALE)

    n_images = len(results)
    for itr in range(n_images):
      viewer.add_labels(**results[itr], scale=pipe.SCALE)
    viewer.window.qt_viewer.view.camera.set_state(past_state)

  viewer.window.add_dock_widget(test_thresholds_gui)
  test_thresholds_gui()

In [ ]:
print(LOW_THRESH, HIGH_THRESH, LOW_AUTO_THRESH, HIGH_AUTO_THRESH)

pipe.segment(LOW_THRESH, HIGH_THRESH, LOW_AUTO_THRESH, HIGH_AUTO_THRESH)

In [ ]:
view_3D({'data': pipe.imsegmented, 'colormap': 'inferno', 'name': 'segmented'},
        {'data': pipe.labels, 'colormap': 'gist_earth', 'gamma': .8, 'name': 'labels'})

### 3.3 Visualize segmented cells to determine cutoff volumes

#### 3.3.1 Check segmented cells on whole image

In [ ]:
pipe.volume_cutoff(500, False)

In [ ]:
somas_estimates_path = 'Autocropped/CONTROL_MSP2.1MB_4_LONG MARK_20X_SEC 1_LEFT HILUS_28 DAYczi-ML/.somas_estimates.npy'
pipe.approximate_somas(src=somas_estimates_path)

# napari clump sep

In [ ]:
pipe.separate_clumps(somas_estimates_path)

In [ ]:
len(pipe.regions)

#### 3.3.2: Check batches of objects

In [ ]:
pipe.show_segmented('grid', 50)

#### 3.3.2: Check individual objects
Select individual objects using `OBJ_INDEX`.

In [ ]:
pipe.show_segmented()

#### 3.3.3: Refine soma approximations
Select individual objects using `OBJ_INDEX`.

In [ ]:
pipe.refine_soma_approx()

---

## Step 4: Export autocropped 3D cells or 2D max intensity projections

Set two parameters:
- `LOW_VOLUME_CUTOFF`: to filter out noise/artifacts
- `HIGH_VOLUME_CUTOFF`: to filter out cell clusters

For choosing between 3D segmented cells or 2D max intensity projections:
- Set `OUTPUT_OPTION` = '3d' for 3D cells, or
- Set `OUTPUT_OPTION` = 'mip' for Max Intensity Projections.

In [ ]:
pipe.export_cropped()

---

## Step 5: Batch all

In [ ]:
import os

ROOT = 'E:/'

SECTIONS = 'Garima Confocal/SAL,DMI, FLX ADN HALO_TREATMENT_28 DAYS/control_28 days'
dir_path = os.path.join(ROOT, SECTIONS)

files = list(map(lambda s: os.path.join(dir_path, s), os.listdir(dir_path)))
IM_EXT = '.czi'
ROI_EXT = '.roi'
REGION = 'HILUS'
ROI_NAME = 'ML'

im_paths = list(filter(lambda f: f.endswith(IM_EXT) and REGION in f.upper(), files))
roi_paths = list(filter(lambda f: f.endswith(ROI_EXT) and REGION in f.upper() and ROI_NAME in f.upper(), files))
somas_estimates_paths = [somas_estimates_path]
somas_estimates_paths.extend([None] * (len(im_paths)-len(somas_estimates_paths)))
list(map(len, (im_paths, roi_paths, somas_estimates_paths)))

In [ ]:
# def batch_init(pipe, im_paths, roi_paths, soma_estimates_paths):
im_paths = iter(im_paths)
roi_paths = iter(roi_paths)
somas_estimates_paths = iter(somas_estimates_paths)
ref_impath = pipe.ref_im_path
ref_roipath = pipe.REF_ROI_PATH
cache_dir = pipe.CACHE_DIR
deconv_itr = pipe.DECONV_ITR
clip_limit = pipe.CLIP_LIMIT
low_thresh = pipe.LOW_THRESH
high_thresh = pipe.HIGH_THRESH
low_auto_thresh = pipe.LOW_AUTO_THRESH
high_auto_thresh = pipe.HIGH_AUTO_THRESH
low_volume_cutoff = pipe.LOW_VOLUME_CUTOFF
high_volume_cutoff = pipe.HIGH_VOLUME_CUTOFF
out_dims = pipe.OUT_DIMS
segment_type = pipe.SEGMENT_TYPE

### Iterations

In [ ]:
im_path, roi_path, somas_estimates_path = next(im_paths), next(roi_paths), next(somas_estimates_paths)
im_path, roi_path, somas_estimates_path

In [ ]:
pipe = ac.TissueImage(im_path, roi_path, ROI_NAME, DECONV_ITR, CLIP_LIMIT,
                      REF_IMAGE, REF_FILE_ROI, cache_dir)
pipe.segment(low_thresh, high_thresh, low_auto_thresh, high_auto_thresh)
pipe.volume_cutoff(low_volume_cutoff, False)
pipe.approximate_somas(src=somas_estimates_path)
pipe.LOW_VOLUME_CUTOFF = low_volume_cutoff
pipe.HIGH_VOLUME_CUTOFF = high_volume_cutoff
pipe.OUT_DIMS = out_dims
pipe.SEGMENT_TYPE = segment_type
pipe.separate_clumps(somas_estimates_path, sync=True)